In [1]:
#Inspiration :D
#https://gitlab.com/ryanlogsdon/pubsub-for-python/-/blob/main/subscriber.py
#https://www.youtube.com/watch?v=xOtrCmPjal8&ab_channel=D-I-Ry
import os
from google.cloud import pubsub_v1
import sys
import time
import pandas as pd
import http.client
import datetime
from io import StringIO

In [2]:
! python --version

Python 3.7.4


## Create connection to PubSub

In [2]:
credentials_path = 'weather-based-forecasting-v2-c4bde37656a7.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path

In [35]:
publisher = pubsub_v1.PublisherClient(
    publisher_options = pubsub_v1.types.PublisherOptions(
        enable_message_ordering=True,
    )
)
topic_path = 'projects/weather-based-forecasting-v2/topics/test_topic'


## Download the data

In [4]:
time.sleep(5)

#locations = ["Berlin", "Hamburg","Bremen"]


conn = http.client.HTTPSConnection("visual-crossing-weather.p.rapidapi.com")

headers = {
    "X-RapidAPI-Key": "c7b48b7c4emsh8a3b688e71b9896p1be4d0jsnda2e5b27b486",
    "X-RapidAPI-Host": "visual-crossing-weather.p.rapidapi.com"
}

conn.request("GET", "/forecast?location=Hamburg&aggregateHours=1&shortColumnNames=0&unitGroup=metric&contentType=csv",
             headers=headers)

res = conn.getresponse()
data = res.read()

In [5]:
s = str(data, 'utf-8')
data_string = StringIO(s)
df = pd.read_csv(data_string)

df2 = df.copy()
df2.index = pd.to_datetime(df2["Date time"])
df2 = df2.asfreq(freq="1min")
df2 = df2.interpolate(method ='linear', limit_direction ='forward')
df2["Conditions"] = df2.Conditions.fillna(method="backfill")
df2["Address"] = df2.Address[0]
df2["Date time"] = df2.index
df2["Resolved Address"] = df2.Name[0]
df2["Name"] = df2.Name[0]

# we're taking 10 days of a forecast
# now = datetime.datetime.now()
# minutes = now.minute
# df3 = df2.iloc[minutes-1:60*24*10, :]
df3 = df2.iloc[:60*24*10, :]
#df3.to_csv(sys.stdout, index=False)

In [8]:
now = datetime.datetime.now()
minutes = now.minute
df3_hour = df3.iloc[minutes-1:minutes+59, :]
df3_hour.head(5)

,Address,Date time,Latitude,Longitude,Resolved Address,Name,Wind Direction,Temperature,Wind Speed,Cloud Cover,Heat Index,Chance Precipitation (%),Precipitation,Sea Level Pressure,Snow Depth,Snow,Relative Humidity,Wind Gust,Wind Chill,Conditions
Date time,,,,,,,,,,,,,,,,,,,,
2022-12-17 12:15:00,Hamburg,2022-12-17 12:15:00,53.5537,9.99183,Hamburg,Hamburg,195.15,-2.275,5.375,13.350000,NaN,0.0,0.0,1026.0,0.0,0.0,84.125000,12.80,-4.400000,Clear
2022-12-17 12:16:00,Hamburg,2022-12-17 12:16:00,53.5537,9.99183,Hamburg,Hamburg,195.34,-2.260,5.400,13.053333,NaN,0.0,0.0,1026.0,0.0,0.0,84.033333,12.86,-4.393333,Clear
2022-12-17 12:17:00,Hamburg,2022-12-17 12:17:00,53.5537,9.99183,Hamburg,Hamburg,195.53,-2.245,5.425,12.756667,NaN,0.0,0.0,1026.0,0.0,0.0,83.941667,12.92,-4.386667,Clear
2022-12-17 12:18:00,Hamburg,2022-12-17 12:18:00,53.5537,9.99183,Hamburg,Hamburg,195.72,-2.230,5.450,12.460000,NaN,0.0,0.0,1026.0,0.0,0.0,83.850000,12.98,-4.380000,Clear
2022-12-17 12:19:00,Hamburg,2022-12-17 12:19:00,53.5537,9.99183,Hamburg,Hamburg,195.91,-2.215,5.475,12.163333,NaN,0.0,0.0,1026.0,0.0,0.0,83.758333,13.04,-4.373333,Clear


In [32]:
a = df3_hour.iloc[i,:].name
int(a.value / 10**9)

1671279360

## Send observations in intervals to PubSub - check Weather_PubSub_check for results

In [41]:
for i in range(2):
    obs = str(df3_hour.iloc[i,:].astype(str).to_dict())
    index = df3_hour.iloc[i,:].name
    index = str(int(index.value / 10**9)).encode("utf-8")
    obs = obs.encode("utf-8")
    future = publisher.publish(topic_path, obs, ordering_key=index)
    print(f'published message id {future.result()}')
    time.sleep(60)

published message id 6600070271448593
published message id 6600094111161820


## Send Forecast in intervals to PubSub

In [34]:
df3_forecast = df3.iloc[minutes-1:, :]
df3_forecast.index = df3_forecast.index.astype(str)
for i in range(60):
    
    df3_forecast_window = str(df3_forecast.iloc[i:i+7*24*60,:].astype(str).to_dict("index"))
#     for key in df3_forecast_window.keys():
#         df3_forecast_window[key] = str(df3_forecast_window[key])
    df3_forecast_window = df3_forecast_window.encode('utf-8')
        
    info = "Weather forecast for Hamburg, {}".format(df3_forecast.index[i])
    info = info.encode('utf-8')
    
    future = publisher.publish(topic_path, df3_forecast_window)
    print(f'published message id {future.result()}')
    time.sleep(60)

published message id 6593353259342791
published message id 6593325870007356
